In [182]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import emoji


In [183]:
#create copy하세요
# 전 copy가 원본이라 생각

In [184]:
#데이터 불러오기 
dataset_path = Path("/Users/jihoyeom/Desktop/projects/zigzag/dataset")
item = "반소매티셔츠"
item_dir = dataset_path/item
og_dir = dataset_path/(item + ' copy')

def get_file_paths(directory):
    files_path = []
    if os.path.exists(directory):
        files_path = [str(file) for file in directory.rglob('*')]
    else:
         print(f"directory not found")
         return
    return files_path

item_paths = get_file_paths(item_dir)
og_paths = get_file_paths(og_dir)


In [185]:
# Helper Fucn
def get_file(path):
    return pd.read_csv(path,on_bad_lines="skip")

def save_file(df, path):
    df.to_csv(path, index=False)

In [193]:
p1 = get_file(item_paths[0])
p2 = get_file(item_paths[1])
p3 =  get_file(item_paths[2])

In [187]:
p1

,아이디,리뷰어 등급,작성 날짜,옵션1,옵션2,상품 사이즈,퀄리티,색감,키,몸무게,사이즈,리뷰내용
0,XV5OYRp0,상위 10%,23.05.18,SKY_BLUE,FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,162cm,60kg,상의 55,네 저는 162에 59~60 입니다 가슴은 80~85 a 이구요 다행히 하체비만이라...
1,fdPnc092,NaN,22.06.28,WHITE,FREE,생각보다 커요,아주 만족해요,화면과 비슷해요,161cm,39kg,상의 33 이하,원영 아가... 생각보다 컷군아...^^ 아이구...\n\n랄라블라 알바생인데 손님...
2,YCmfoX15,NaN,22.06.13,WHITE,FREE,정사이즈예요,보통이에요,화면과 비슷해요,157cm,50kg,상의 55,좀 어깡이 되버리긴 하지만 \n재질이랑 마감좋고 맘에들었어요!\n근데 다림질 그냥 ...
3,bnrq2khv,NaN,22.06.11,WHITE,FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,164cm,46kg,상의 44,제가 이걸 입는다고 워녕이가 되지는 않지만... 네 나름 만족합니다 밴딩 안 들어간...
4,eovbR2Kz,NaN,22.06.02,WHITE,FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,167cm,56kg,상의 55,"제가 딱 좋아하는 기장&핏이라서 넘 마음에 들어요!! 어깨도 잘 잡혀있고, 흐물거리..."
...,...,...,...,...,...,...,...,...,...,...,...,...
995,8OYflGjv,NaN,21.09.10,WHITE,FREE,정사이즈예요,보통이에요,화면과 비슷해요,164cm,51kg,상의 55,단추구멍 마감이 조금 아쉬웠어요 그래도 전체적으로 핏이 예뻐서 자주 입을것 같아요
996,yEeJJEaS,NaN,24.04.29,BLACK,FREE,정사이즈예요,보통이에요,화면과 비슷해요,167cm,53kg,상의 55,블랙은 완전 강추보단 그냥 살짝 추천 정도? 화면이랑 거의 비슷하고 굉장히 크롭해요
997,nN0BDWDJ,NaN,23.07.18,SKY_BLUE,FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,172cm,68kg,상의 66,키가 큰 저에겐 조금 짧지만 그래더 매우 만족입니당 깔끔해보이고 부해보이지도 않아요...
998,eun9*****,상위 15%,23.06.23,SKY_BLUE,FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,156cm,50kg,상의 55,크롭 셔츠 사구싶었는데 예쁜 쨍한색감에 재질도 넘 좋구 허리선 잡아주면서 편안합미난


In [188]:
# 수동적으로 해야할 것들
# Check for 키, 몸무게, 사이즈 rows where the unit is missing or invalid

unit_dict = {'키': 'cm', '몸무게': 'kg', '사이즈': '상의'}

def check_wrong_data(df):
    invalid_rows_list = []
    
    for col in unit_dict.keys():
        expected_unit = unit_dict[col]
        invalid_rows = df[df[col].apply(lambda x: pd.notna(x) and isinstance(x, str) and expected_unit not in x)]
        invalid_rows_list.append(invalid_rows)
    
    # Combine all invalid rows and drop duplicates
    consolidated_invalid_rows = pd.concat(invalid_rows_list).drop_duplicates()
    
    return consolidated_invalid_rows

def check_wrong_items(option):
    if option == "og":
        paths = og_paths
    elif option == "replace":
        paths = item_paths
    else:
        return (f"no option exists")
    combined_df = pd.DataFrame()
    
    for path in paths:
        if path.endswith('.txt'):
            continue
        df = get_file(path)
        wrong_df = check_wrong_data(df)
        combined_df = pd.concat([combined_df, wrong_df], ignore_index=True)
    return combined_df


In [189]:
before = check_wrong_items("og")
before

,아이디,리뷰어 등급,작성 날짜,옵션1,옵션2,상품 사이즈,퀄리티,색감,키,몸무게,사이즈,리뷰내용
0,P7b9j1lU,NaN,24.08.07,기본 블랙(black),3[88~99],정사이즈예요,보통이에요,화면과 비슷해요,75kg,NaN,NaN,작을까봐 걱정 했는데 괜한 걱정 이였습니다 ㅋㅋㅋ 완전 좋아요!! 진짜 사길 잘 한...
1,iLcVm1hL,NaN,24.08.29,기본 블랙(black),3[88~99],정사이즈예요,보통이에요,화면과 비슷해요,168cm,상의 88,NaN,사실 이게 넥 라인이 많이 벌어져 있어서 팔뚝살 많은 저한테는 솔직히 어깡처럼 보여...
2,YrjUoZ4b,NaN,24.07.28,기본 아이보리(ivory),3[88~99],정사이즈예요,보통이에요,화면과 비슷해요,160cm,상의 55,NaN,참나....예뻐요 미드 크신 분들은 부담스러울수도.. 저는 전혀 부담스럽지않았어요 ...
3,YrjUoZ4b,NaN,24.07.28,기본 아이보리(ivory),2[66~77],정사이즈예요,보통이에요,화면과 비슷해요,160cm,상의 55,NaN,예쁘구\n....굿...좋아요..굿...굿....
4,aph03cgs,NaN,24.10.19,Blue Gray,Free,정사이즈예요,아주 만족해요,화면과 비슷해요,158cm,상의 66,NaN,오랜만에 오프숄더가 입고싶어서 과하지 않은 걸로 찾다가 구매했는데 언발란스라 부담스...
5,mmjSbHCk,NaN,24.08.02,Black,Free,정사이즈예요,아주 만족해요,화면과 비슷해요,160cm,상의 55,NaN,와~~~~여행전날 급하게 주문했는데...\n200%만족이에요~~\n두꺼운 팔뚝도 가...
6,Onj80AYZ,상위 15%,24.08.15,Blue Gray,Free,정사이즈예요,아주 만족해요,화면과 비슷해요,160cm,상의 55,NaN,가격대비 너무 만족도 높은 상품이에요 \n한쪽으로 어깨선내리면 라인감이 부담스럽지않...
7,pnqQWZps,NaN,24.07.05,Black,Free,정사이즈예요,아주 만족해요,화면과 비슷해요,151cm,상의 55,NaN,팔뚝도 있고 어깨도 있어서 오프숄더를 입지 않는 피플인데 약간 한쪽 팔뚝 감싸주기도...
8,xm6lSXTA,상위 20%,24.12.20,아이보리,NaN,생각보다 커요,아주 만족해요,화면과 비슷해요,164cm,상의 55,NaN,회색을 여름에 사고 입고다니다가 재질이 너무 탄탄하고 만족스러워서 흰색도 구매했어요...
9,6ckPWryc,NaN,24.06.18,메란지그레이,NaN,정사이즈예요,보통이에요,화면과 비슷해요,163cm,상의 55,NaN,기본티셔츠로 다른색상이랑 같이 구매했는데 핏이랑 전체적으로 너무 만족스럽습니당!! ...


In [190]:
# 위 두개를 돌려보고 난 뒤
# 긴소매 기준 , unit 기준으로 대체해도 괜찮다고 판단
def move_data(df):
    for index, row in df.iterrows():
        for col in unit_dict.keys():
            expected_unit = unit_dict[col]
            if pd.notna(row[col]) and expected_unit not in str(row[col]):
                # Find the correct column for this value
                for target_col, target_unit in unit_dict.items():
                    if target_unit in str(row[col]):
                        # Move the value to the correct column
                        df.at[index, target_col] = row[col]
                        df.at[index, col] = None  # Clear the original value
                        break
    return df

def move_all_data(option):
    if option == "og":
        paths = og_paths
    elif option == "replace":
        paths = item_paths
    else:
        return (f"no option exists")
    
    for path in item_paths:
        if path.endswith('.txt'):
            continue
        df = get_file(path)
        updated_df = move_data(df) # 여기서 실제 inplace
        save_file(updated_df, path)
    
    


In [192]:
# For Sanity Check
move_all_data("replace") #og 가 아닌 것에 inplace
after = check_wrong_items("replace")
after

,아이디,리뷰어 등급,작성 날짜,옵션1,옵션2,상품 사이즈,퀄리티,색감,키,몸무게,사이즈,리뷰내용


In [134]:
# Check if any rows are NaN
def remove_all_NaN_row(df):
    df= df.dropna(how='all')
    return df
    
# Remove units and spaces 
def drop_units(df):
    df["키"] = df["키"].str.replace(r"\s*cm\s*","",regex = True) 
    df["몸무게"] = df["몸무게"].str.replace(r"\s*kg\s*","",regex = True)
    df["사이즈"] = df["사이즈"].str.replace(r"\s*상의\s*","",regex = True)
    return df


# Change it to float
def change_to_int(df,col):
    if len(col) > 1:
        for c in col:
            #<NA>로 되버려 pandas._libs.missing.NAType
            df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")
    else:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
    return df


# Remove emoji
def remove_emoji(df):
    def remove_emojis(text):
        return emoji.replace_emoji(text, replace='')  
        df["리뷰"] =df["리뷰"].apply(remove_emojis)
    return df


In [ ]:
p1.descibe()

,아이디,리뷰어 등급,작성 날짜,옵션1,옵션2,상품 사이즈,퀄리티,색감,키,몸무게,사이즈,리뷰내용
count,1000,180,1000,1000,1000,1000,1000,999,903,903,903,1000
unique,979,8,409,4,1,3,3,3,29,43,7,994
top,FaMpM90N,상위 5%,22.05.11,WHITE,FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,160cm,50kg,상의 55,색상만 다르게 두 제품 구매했는데 이건 넉넉하고 흰색은 터질 것 같아요 검수 좀 제...
freq,3,37,15,850,1000,722,657,970,117,114,495,2


In [206]:
p1["작성 날짜"]

0      23.05.18
1      22.06.28
2      22.06.13
3      22.06.11
4      22.06.02
         ...   
995    21.09.10
996    24.04.29
997    23.07.18
998    23.06.23
999    23.06.06
Name: 작성 날짜, Length: 1000, dtype: object

In [204]:
p1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   아이디     1000 non-null   object
 1   리뷰어 등급  180 non-null    object
 2   작성 날짜   1000 non-null   object
 3   옵션1     1000 non-null   object
 4   옵션2     1000 non-null   object
 5   상품 사이즈  1000 non-null   object
 6   퀄리티     1000 non-null   object
 7   색감      999 non-null    object
 8   키       903 non-null    object
 9   몸무게     903 non-null    object
 10  사이즈     903 non-null    object
 11  리뷰내용    1000 non-null   object
dtypes: object(12)
memory usage: 93.9+ KB
